# Criando uma imagem de container customizada para o SageMaker

## Especifique a sua imagem de container

### Como o Amazon SageMaker executa o seu container

Como você pode executar a mesma imagem no treinamento ou na hospedagem, o Amazon SageMaker executa o seu container com o argumento `train` ou `serve`. Como o container processa esse argumento depende da sua implementação:

* No exemplo aqui, não definimos um `ENTRYPOINT` no Dockerfile, portanto o SageMaker executará o comando `train` no momento do treinamento e `serve` no momento da hospedagem. Neste exemplo, definimos esses comandos como scripts executáveis em Python, mas poderiam ser qualquer programa que desejamos iniciar nesse ambiente.
* Se você especificar um programa como um `ENTRYPOINT` no Dockerfile, esse programa será executado na inicialização e o seu primeiro argumento será `train` ou `serve`. O programa pode então verificar esse argumento e decidir o que fazer.
* Se você estiver criando containers separados para treinamento e hospedagem (ou criando apenas para um deles), você pode definir um programa como um `ENTRYPOINT` no Dockerfile e ignorar (ou verificar) o primeiro argumento passado.

In [38]:
!cat container/Dockerfile

FROM ubuntu:18.04

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

RUN pip --no-cache-dir install boto3 numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

COPY dummy /opt/program
WORKDIR /opt/program


### Efetuando o build da imagem e push para o [Amazon Elastic Container Registry](https://aws.amazon.com/pt/ecr/)

In [40]:
%%sh

# Nome da imagem
algorithm_name=sagemaker-dummy

cd container

chmod +x dummy/train
chmod +x dummy/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Define a região padrão como us-east-1 caso não especificada na configuração do AWS CLI
region=$(aws configure get region)
region=${region:-us-east-1}

# Verifique se já existe um repositório no ECR
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Efetua o build da imagem localmente
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

# Envia imagem para o ECR
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}
docker push ${fullname}

#1 [internal] load build definition from Dockerfile
#1 sha256:aab511f59e57e986a282963b32585c7edb02d5ac4e795d4e8f0dbd43fc92ae84
#1 transferring dockerfile: 121B 0.0s done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 sha256:a54ec23d5a0802fde75c2789be3b5d14cd78b686ab827af992ee68e83e004fe2
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 DONE 2.5s

#4 [1/7] FROM docker.io/library/ubuntu:18.04@sha256:152dc042452c496007f07ca9127571cb9c29697f42acbfad72324b2bb2e43c98
#4 sha256:0874120c9a032c9cafa396e6559d095de2b8b000b243455626d71bbf8a51cdb3
#4 DONE 0.0s

#9 [internal] load build context
#9 sha256:4e275aca27f5754f9c438b034929c19643d14a074ad369b137df0f58a0ff6d72
#9 transferring context: 3.92kB done
#9 DONE 0.0s

#6 [3/7] RUN ln -s /usr/bin/python3 /usr/bin/python
#6 sha256:3f837c0dca010b7fc9dbc346c55b1f366559c55b9a1a7ff3a75c707a06cccecd
#6 CACHED

#7 [4/

Login Succeeded
The push refers to repository [163701031472.dkr.ecr.us-east-1.amazonaws.com/sagemaker-dummy]
5f70bf18a086: Preparing
68ce0b97498a: Preparing
720449616670: Preparing
08092c284113: Preparing
260368bf0952: Preparing
12b23572dd9e: Preparing
548a79621a42: Preparing
12b23572dd9e: Waiting
548a79621a42: Waiting
08092c284113: Layer already exists
720449616670: Layer already exists
5f70bf18a086: Layer already exists
12b23572dd9e: Layer already exists
548a79621a42: Layer already exists
260368bf0952: Layer already exists
68ce0b97498a: Pushed
latest: digest: sha256:5a862cb80dc9baf91adf03056d840e6521ef320d42d69d919daa526019c41f27 size: 1781


## Testando local ou no notebook do SageMaker Studio

No diretório `container/local_test` há alguns scripts para testar local.

Os scripts são:
* `train_local.sh`: Execute este script com o nome da imagem e ele executará o treinamento localmente. Por exemplo, você pode executar `$ ./train_local.sh sagemaker-dummy`. Ele gerará um modelo no diretório `/test_dir/model`. Você precisará modificar o diretório `test_dir/input/data/...` para configurá-lo com os canais e dados corretos para o seu algoritmo. Além disso, você deverá modificar o arquivo `input/config/hyperparameters.json` para definir as configurações de hiperparâmetros que deseja testar.
* `serve_local.sh`: Execute este script com o nome da imagem depois de treinar o modelo e ele deverá servir o modelo. Por exemplo, você pode executar `$ ./serve_local.sh sagemaker-dummy`. Ele será executado e aguardará por solicitações. Basta interrompê-lo usando o comando de interrupção do teclado.
* `predict.sh`: Execute este script com o nome de um arquivo (payload) e (opcionalmente) o tipo de conteúdo HTTP desejado. O tipo de conteúdo será definido como `text/csv` por padrão. Por exemplo, você pode executar `$ ./predict.sh payload.csv text/csv`.

## Usando a imagem no SageMaker

In [42]:
import boto3
import sagemaker as sage

role = "CRIAR_ROLE_PARA_O_SAGEMAKER"
sess = sage.Session()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


### Criando um estimator com a imagem customizada

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [43]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
algorithm_name = "sagemaker-dummy"
image = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name)

dummy = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c5.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess
)

dummy.fit(data_location)

INFO:sagemaker:Creating training-job with name: sagemaker-dummy-2023-08-16-21-47-34-498


2023-08-16 21:47:35 Starting - Starting the training job...
2023-08-16 21:47:51 Starting - Preparing the instances for training......
2023-08-16 21:49:09 Downloading - Downloading input data
2023-08-16 21:49:09 Training - Training image download completed. Training in progress....
2023-08-16 21:49:30 Uploading - Uploading generated training model
2023-08-16 21:49:30 Completed - Training job completed
..Training seconds: 42
Billable seconds: 42


In [44]:
training_job_id = "JOB_NAME_DO_ULTIMO_TREINO"
training_job = sess.describe_training_job(training_job_id)
training_job

{'TrainingJobName': 'sagemaker-dummy-2023-08-16-21-47-34-498',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:163701031472:training-job/sagemaker-dummy-2023-08-16-21-47-34-498',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-163701031472/output/sagemaker-dummy-2023-08-16-21-47-34-498/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'AlgorithmSpecification': {'TrainingImage': '163701031472.dkr.ecr.us-east-1.amazonaws.com/sagemaker-dummy:latest',
  'TrainingInputMode': 'File',
  'EnableSageMakerMetricsTimeSeries': False},
 'RoleArn': 'arn:aws:iam::163701031472:role/sagemaker-role',
 'InputDataConfig': [{'ChannelName': 'training',
   'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://sagemaker-us-east-1-163701031472/DEMO-dummy',
     'S3DataDistributionType': 'FullyReplicated'}},
   'CompressionType': 'None',
   'RecordWrapperType': 'None'}],
 'OutputDataConfig': {'KmsKeyId': '',
  'S3OutputPath': 's

## Deploy do modelo treinado com a imagem customizada

In [48]:
# Caso esteja na mesma sessão que o treinamento podemos utilizar o próprio objeto do estimator para fazer o deploy
# Carregando os dados do estimator caso saia da sessão
dummy = sage.estimator.Estimator.attach(training_job_id)
predictor = dummy.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials



2023-08-16 21:49:30 Starting - Preparing the instances for training
2023-08-16 21:49:30 Downloading - Downloading input data
2023-08-16 21:49:30 Training - Training image download completed. Training in progress.
2023-08-16 21:49:30 Uploading - Uploading generated training model
2023-08-16 21:49:30 Completed - Training job completed


INFO:sagemaker:Creating model with name: sagemaker-dummy-2023-08-16-21-56-39-173
INFO:sagemaker:Creating endpoint-config with name sagemaker-dummy-2023-08-16-21-56-39-173
INFO:sagemaker:Creating endpoint with name sagemaker-dummy-2023-08-16-21-56-39-173


---!

In [50]:
predictor.predict("test")

b'teste dummy'